In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import accuracy_score
from segysak.segy import segy_loader
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import cv2 as cv
import warnings
import pickle
import segyio
import xarray
import os

warnings.filterwarnings(action='ignore')

In [ ]:
def norm_data(X, x_min, x_max):
    return (X - x_min) / (x_max - x_min)

def obj_to_pixel_df(cube_sgy, obj, width, height, index_label):
    """
    Function to read the ".obj" file that contains the coordinates
    of the facie's selected points and return a DataFrame that contains
    the pixels around each point.
    
    Paramaters
    ----------
    cube_sgy : segyio.segy.SegyFile or xarray.core.dataset.Dataset
        Loaded ".sgy" file.
    obj : list
        Loaded object file.
    width : int
        Size of window width.
    height : int
        Size of window height.
    index_label : str
        Label of the dataset index.
        
    Returns
    -------
    dataset : pandas.DataFrame
        Each row of the DataFrame represents a window and each column
        represents one pixel of that window.
    """
    
    dataset = pd.DataFrame()

    if width % 2 > 0:
        width_coef = 1
    else:
        width_coef = 0
    
    if height % 2 > 0:
        height_coef = 1
    else:
        height_coef = 0

    half_width = int(round(width / 2 - 0.5, 0))
    half_height = int(round(height / 2 - 0.5, 0))
    
    count = 0
    for click in obj:
        iline_number = click['inline']
        xline_number = click['crossline']

        x = click['x']
        y = click['y']

        if type(cube_sgy) == xarray.core.dataset.Dataset:
            if iline_number != None:
                img = cube_sgy.isel(iline=iline_number).data.T.to_numpy()[x - half_width : x + half_width + width_coef, y - half_height : y + half_height + height_coef]
            else:
                img = cube_sgy.isel(xline=xline_number).data.T.to_numpy()[x - half_width : x + half_width + width_coef, y - half_height : y + half_height + height_coef]
        elif type(cube_sgy) == segyio.segy.SegyFile:
            if iline_number != None:
                img = cube_sgy.iline[iline_number][x - half_width : x + half_width + width_coef, y - half_height : y + half_height + height_coef].T
            else:
                img = cube_sgy.xline[xline_number][x - half_width : x + half_width + width_coef, y - half_height : y + half_height + height_coef].T

        pixels = img.reshape(-1,1)

        if len(pixels) == int(width * height):
            pixels = norm_data(pixels, np.min(pixels), np.max(pixels))
            dataset[f'{index_label}{count}'] = [pixel[0] for pixel in pixels.tolist()]

        count += 1

    dataset = dataset.T
    #dataset.index = [f'{index_label}{i}' for i in range(len(dataset))]

    if index_label == 'Fault':
        dataset['y'] = [1 for i in range(len(dataset))]

    elif index_label == 'Non_Fault':
        dataset['y'] = [0 for i in range(len(dataset))]

    return dataset

def get_df_from_dats(path):
    fault_list = [fault for fault in os.listdir(path) if fault.endswith('.dat')]

    df = pd.DataFrame()

    for fault in fault_list:
        df_to_concat = pd.read_csv(f'{path}\{fault}', delim_whitespace=True, header=None)
        df_to_concat.columns = ['Name', 'Inline', 'Xline', 'Time', 'idk', 'n']

        i_or_x = fault.split('_')[1]

        df_to_concat['Line'] = [i_or_x for _ in range(len(df_to_concat))]

        df = pd.concat([df, df_to_concat])

    df['Time'] /= 4
    df['Time'] = round(df['Time'])

    id = []
    count = -1
    for i in range(len(df)):
        if df['n'].iloc[i] == 0:
            count += 1
        id += [count]
    df['ID'] = id

    df.drop(['Name', 'idk', 'n'], axis=1, inplace=True)

    df = df[df.columns[::-1]]

    df.set_index('ID', inplace=True)

    return df

def plot_faults(file_path, cube_path, cmap='gray', linewidth=2):

    df = pd.read_csv(file_path, delim_whitespace=True)
    cube = segyio.open(cube_path)
    
    iline_list = cube.ilines
    xline_list = cube.xlines
    iline0 = iline_list[0]
    xline0 = xline_list[0]

    print(f'Initial Values:\n   Inline: {iline0} | Crossline: {xline0}')

    rows = len(df['Inline'][df['Line'] == 'Inline'].unique()) + len(df['Xline'][df['Line'] == 'Xline'].unique())
    cols = 1

    fig, ax = plt.subplots(rows, cols, figsize=(15 * cols, 7 * rows))
    count = 0
    for line_str in df['Line'].unique():
        for line_number in df[line_str][df['Line'] == line_str].unique():
            img = cube.iline[line_number].T if line_str == 'Inline' else cube.xline[line_number].T
            mask = np.zeros(img.shape) * np.nan

            X = df['Xline'][df[line_str] == line_number][df['Line'] == line_str] - xline0 if line_str == 'Inline' else df['Inline'][df[line_str] == line_number][df['Line'] == line_str] - iline0
            Y = df['Time'][df[line_str] == line_number][df['Line'] == line_str]

            ax[count].imshow(img, cmap=cmap)
            ax[count].set_title(f'{line_str}: {line_number}')
            if line_str == 'Inline':
                for id in df['ID'][df['Line'] == line_str][df['Inline'] == line_number].unique():
                    [cv.line(mask, (int(df['Xline'][df['ID'] == id].iloc[i-1] - xline0), int(df['Time'][df['ID'] == id].iloc[i-1])), (int(df['Xline'][df['ID'] == id].iloc[i] - xline0), int(df['Time'][df['ID'] == id].iloc[i])), (1,0,0), linewidth) for i in range(1, len(df['ID'][df['ID'] == id]))]
                ax[count].set_xlabel('Xline')
            else:
                for id in df['ID'][df['Line'] == line_str][df['Xline'] == line_number].unique():
                    [cv.line(mask, (int(df['Inline'][df['ID'] == id].iloc[i-1] - xline0), int(df['Time'][df['ID'] == id].iloc[i-1])), (int(df['Inline'][df['ID'] == id].iloc[i] - xline0), int(df['Time'][df['ID'] == id].iloc[i])), (1,0,0), linewidth) for i in range(1, len(df['ID'][df['ID'] == id]))]
                ax[count].set_xlabel('Inline')
            ax[count].imshow(mask)
            ax[count].set_ylabel('Time')
            count += 1
            
def get_amplitude_mask_dataset(file_path, cube, width=15, height=15, linewidth=2, x_type='float32', y_type='int8'):

    df = pd.read_csv(file_path, delim_whitespace=True)

    iline_list = cube.ilines
    xline_list = cube.xlines
    iline0 = iline_list[0]
    xline0 = xline_list[0]

    X = {}
    y = {}
    fault_or_not = {}
    
    n_occurrences = height * linewidth / 2

    count = 0
    for line_str in df['Line'].unique():
        for line_number in df[line_str][df['Line'] == line_str].unique():
            print(line_str, " -> ", line_number, " -> ", count)
            img = cube.iline[line_number].T if line_str == 'Inline' else cube.xline[line_number].T
            mask = np.zeros(img.shape)

            if line_str == 'Inline':
                for id in df['ID'][df['Line'] == line_str][df['Inline'] == line_number].unique():
                    [cv.line(mask, (int(df['Xline'][df['ID'] == id].iloc[i-1] - xline0), int(df['Time'][df['ID'] == id].iloc[i-1])), (int(df['Xline'][df['ID'] == id].iloc[i] - xline0), int(df['Time'][df['ID'] == id].iloc[i])), (1,0,0), linewidth) for i in range(1, len(df['ID'][df['ID'] == id]))]
            else:
                for id in df['ID'][df['Line'] == line_str][df['Xline'] == line_number].unique():
                    [cv.line(mask, (int(df['Inline'][df['ID'] == id].iloc[i-1] - xline0), int(df['Time'][df['ID'] == id].iloc[i-1])), (int(df['Inline'][df['ID'] == id].iloc[i] - xline0), int(df['Time'][df['ID'] == id].iloc[i])), (1,0,0), linewidth) for i in range(1, len(df['ID'][df['ID'] == id]))]

            for i in range(img.shape[0] - (height - 1)):
                for j in range(img.shape[1] - (width - 1)):

                    window_amplitude = img[i : i + height, j : j + width]
                    window_amplitude = norm_data(window_amplitude, np.min(window_amplitude), np.max(window_amplitude))

                    window_mask = mask[i : i + height, j : j + width]

                    X.update({f'W_{count}' : window_amplitude.reshape(1,-1)[0]})
                    y.update({f'W_{count}' : window_mask.reshape(1,-1)[0]})
                    if np.count_nonzero(window_mask == 1) >= n_occurrences:
                        fault_or_not.update({f'W_{count}' : [1]})
                    else:
                        fault_or_not.update({f'W_{count}' : [0]})

                    count += 1
    
    dataset = pd.concat(
        [pd.DataFrame(X).T, pd.DataFrame(y).T.astype('int8'), pd.DataFrame(fault_or_not).T.astype('int8')],
        axis=1)
    renamed_columns = [f'X{i}' for i in range(width * height)] + [f'y{i}' for i in range(width * height)] + ['Fault_or_Not']
    dataset.columns = renamed_columns
    
    return dataset

In [ ]:
#amplitude_cube_path = r'C:\Users\jpgom\Documents\Jão\VS_Code\IC\Seismic_data_w_null.sgy'
#amplitude_cube_path = '/home/gaia/jpedro/Seismic_data_w_null.sgy'
amplitude_cube_path = r'C:\Users\jpg\Desktop\code\Seismic_data_w_null.sgy'

#similarity_cube_path = r'C:\Users\jpgom\Documents\Jão\VS_Code\IC\F3_Similaridade_Full (455).sgy'
#similarity_cube_path = '/home/gaia/jpedro/F3_Similaridade_Full (455).sgy'

amplitude_cube = segyio.open(amplitude_cube_path)
#amplitude_cube = segy_loader(amplitude_cube_path, iline=189, xline=193, cdpx=181, cdpy=185)

### GET X AND Y FROM ".OBJ" FILES

In [ ]:
#file = open(r'C:\Users\jpgom\Documents\Jão\git\facies_selector\Fault.obj', 'rb')
#file = open('/home/gaia/jpedro/git/facies_selector/Fault.obj', 'rb')
file = open(r'C:\Users\jpg\Desktop\code\Fault.obj', mode='rb')
fault_obj = pickle.load(file)
file.close()

#file = open(r'C:\Users\jpgom\Documents\Jão\git\facies_selector\Non_Fault.obj', 'rb')
#file = open('/home/gaia/jpedro/git/facies_selector/Fault.obj', 'rb')
file = open(r'C:\Users\jpg\Desktop\code\Non_Fault.obj', mode='rb')
non_fault_obj = pickle.load(file)
file.close()

width = 13
height = 13

X_fault_amplitude = obj_to_pixel_df('default', amplitude_cube, fault_obj, width, height, 'Fault')
X_non_fault_amplitude = obj_to_pixel_df('default', amplitude_cube, non_fault_obj, width, height, 'Non_Fault')

dataset_amplitude = pd.concat([X_fault_amplitude, X_non_fault_amplitude])
dataset_amplitude.rename(columns={i : f'Pixel{i}_D' for i in range(len(dataset_amplitude.columns) - 1)}, inplace=True)

dataset = dataset_amplitude.dropna()
dataset = dataset.sample(frac=1)

X = dataset.iloc[:,:-1]
y = dataset['y']

# index = 45
# img = np.array(X.iloc[index, :]).reshape(width, height)
# plt.imshow(img, cmap='gray_r', aspect='auto')
# plt.title(X.index[index])

### GET X AND Y FROM ".DAT" FILE

In [ ]:
faults_path = r'C:\Users\jpg\Desktop\code\facies_classification\Faults.dat'
cube_path = r'C:\Users\jpg\Desktop\code\Seismic_data_w_null.sgy'

cube = segyio.open(cube_path)

width = 30
height = 30

dataset = get_amplitude_mask_dataset(faults_path, cube, width, height, linewidth=2, x_type='float32', y_type='int8')

In [ ]:
X = dataset.iloc[ : 399226, : width*height]
y = dataset.iloc[ : 399226, -1]

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=9, train_size=0.70)

print(f'Shapes: \n    X: {X.shape} | y: {y.shape} | X_train: {X_train.shape} | X_test: {X_test.shape} | y_train: {y_train.shape} | y_test: {y_test.shape}')

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=9, train_size=0.70)

In [ ]:
clf = LogisticRegression(penalty='none', 
                          solver='saga',
                          n_jobs=-1,
                          multi_class='multinomial').fit(X_train, y_train)

clf.coef_.shape

y_pred_test = clf.predict(X_test)
acc_test = accuracy_score(y_test, y_pred_test)

y_pred_train = clf.predict(X_train)
acc_train = accuracy_score(y_train, y_pred_train)

print(f'Model Accuracy (Train): {round(acc_train * 100, 2)}% | Model Accuracy (Test): {round(acc_test * 100, 2)}%')

In [ ]:
plot_confusion_matrix(clf, X_test, y_test, display_labels=['Non-Fault', 'Fault'])

In [ ]:
#print(classification_report(y_test,y_pred_test))

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train,y_train)

y_pred_test = knn.predict(X_test)
acc_test = accuracy_score(y_test, y_pred_test)

y_pred_train = knn.predict(X_train)
acc_train = accuracy_score(y_train, y_pred_train)

print(f'Model Accuracy (Train): {round(acc_train * 100, 2)}% | Model Accuracy (Test): {round(acc_test * 100, 2)}%')

In [ ]:
plot_confusion_matrix(knn, X_test, y_test, display_labels=['Non-Fault', 'Fault'])

In [ ]:
#print(classification_report(y_test,y_pred_test))

In [ ]:
acc_k_train = []
acc_k_test = []

for k in range(3,51,2):
   knn = KNeighborsClassifier(n_neighbors=k)
   knn.fit(X_train,y_train)

   y_pred = knn.predict(X_test)
   acc = accuracy_score(y_test, y_pred)

   y_pred_train = knn.predict(X_train)
   acc_train = accuracy_score(y_train, y_pred_train)

   acc_k_train += [acc_train]
   acc_k_test += [acc]


In [ ]:
plt.plot(range(3,51,2), acc_k_train,marker='o')
plt.plot(range(3,51,2), acc_k_test,marker='o')

plt.xticks(np.arange(0,100))
plt.xlim(3,25)
plt.grid()

In [ ]:
# from sklearn.decomposition import PCA
# from sklearn.preprocessing import StandardScaler

In [ ]:
# pca = PCA()

In [ ]:
# scaler = StandardScaler()

# pca.fit(scaler.fit_transform(X_train[:17*17]))

In [ ]:
# X_pca = pca.transform(scaler.fit_transform(X_train[:17*17]))

In [ ]:
# plt.imshow(X_train.iloc[15].values[:17*17].reshape(17,17),interpolation='spline36',cmap='seismic')

In [ ]:
# plt.imshow(X_pca[15][:100].reshape(10,10),interpolation='spline36',cmap='seismic')

In [ ]:
# plt.plot(np.cumsum(pca.explained_variance_ratio_))

In [ ]:
# np.cumsum(pca.explained_variance_ratio_)[100]

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import plot_tree

In [ ]:
dt = DecisionTreeClassifier(criterion='entropy', max_depth=15)
dt.fit(X_train, y_train)

y_pred_test = dt.predict(X_test)
acc_test = accuracy_score(y_test, y_pred_test)

y_pred_train = dt.predict(X_train)
acc_train = accuracy_score(y_train, y_pred_train)

print(f'Model Accuracy (Train): {round(acc_train * 100, 2)}% | Model Accuracy (Test): {round(acc_test * 100, 2)}%')

In [ ]:
plot_confusion_matrix(dt, X_test, y_test, display_labels=['Non-Fault', 'Fault'])

In [ ]:
#print(classification_report(y_test, y_pred_test))

In [ ]:
# plt.figure(figsize=(60,60))
# plot_tree(dt, filled=True, rounded=True, class_names=['Non-Fault', 'Fault'], feature_names=X_train.columns)

In [ ]:
depth_range = range(3, 31, 2)
width_range = range(11, 32, 2)

plt.figure(figsize=(15,10))

for width in width_range:
    height = width

    X_fault = obj_to_pixel_df('amplitude', amplitude_cube, fault_obj, width, height, 'Fault')
    X_non_fault = obj_to_pixel_df('amplitude', amplitude_cube, non_fault_obj, width, height, 'Non_Fault')
    
    dataset = pd.concat([X_fault, X_non_fault])

    dataset.rename(columns={i : f'Pixel{i}' for i in range(len(dataset.columns) - 1)}, inplace=True)

    dataset = dataset.sample(frac=1)

    X = dataset.iloc[:,:-1]
    y = dataset['y']

    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7)

    acc_test_list = []
    acc_train_list = []
    for depth in depth_range:
        acc_test_mean_list = []
        acc_train_mean_list = []

        for i in range(5):
            model = DecisionTreeClassifier(criterion='entropy', max_depth=depth)
            model.fit(X_train, y_train)

            y_pred = model.predict(X_test)
            acc_test = accuracy_score(y_test, y_pred)

            y_pred_train = model.predict(X_train)
            acc_train = accuracy_score(y_train, y_pred_train)

            acc_test_mean_list += [acc_test * 100]
            acc_train_mean_list += [acc_train * 100]

        acc_test_list += [np.mean(acc_test_mean_list)]
        acc_train_list += [np.mean(acc_train_mean_list)]

    plt.plot(depth_range, acc_test_list, label=f'Teste | w_h: {width} x {height}')
    #plt.plot(depth_range, acc_train_list, label=f'Treino | w_h: {width} x {height}')

plt.xlabel('Profundidade Máxima')
plt.ylabel('Precisão (%)')
plt.legend()

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rfc = RandomForestClassifier(n_estimators=100)
rfc.fit(X_train, y_train)

y_pred_test = rfc.predict(X_test)
acc_test = accuracy_score(y_test, y_pred_test)

y_pred_train = rfc.predict(X_train)
acc_train = accuracy_score(y_train, y_pred_train)

print(f'Model Accuracy (Train): {round(acc_train * 100, 2)}% | Model Accuracy (Test): {round(acc_test * 100, 2)}%')

In [ ]:
plot_confusion_matrix(rfc, X_test, y_test, display_labels=['Non-Fault', 'Fault'])

VARREDURA

In [ ]:
iline_number = 300
width, height = 15, 15

iline_amplitude_array = amplitude_cube.iline[iline_number].T
#iline_similarity_array = similarity_cube.isel(iline=iline_number).data.T.to_numpy()

mask_array = np.zeros(iline_amplitude_array.shape) * np.nan

for i, row in enumerate(iline_amplitude_array):
        
    if i > 0 and X_to_predict.shape != (1, width * height):
        pass
        
    for j, col in enumerate(row):
            
        window_amplitude_array = iline_amplitude_array[i : i + height, j : j + width]
        window_amplitude_array = norm_data(window_amplitude_array, np.min(window_amplitude_array), np.max(window_amplitude_array))

        #window_similarity_array = iline_amplitude_array[i : i + height, j : j + width]
        #window_similarity_array = norm_data(window_similarity_array, np.min(window_similarity_array), np.max(window_similarity_array))

        window_center_coord = (i + int(height / 2), j + int(height / 2))
        
        amplitude_pixels = window_amplitude_array.reshape(1,-1)
        #similarity_pixels = window_similarity_array.reshape(1,-1)
        
        X_to_predict = amplitude_pixels
        
        if X_to_predict.shape != (1, width * height):
            pass
        else:
            predicted_value = knn.predict(X_to_predict)[0]

            mask_array[i][j] = predicted_value

In [ ]:
plt.figure(figsize=(17,10))
plt.imshow(mask_array, cmap='gray_r')

In [ ]:
np.linspace(0,100,2000)

In [ ]:
a = {'w1':[1,2,3]}
b = {'w1':[4,5,6]}
c = {'w1':[7,8,9]}
pd.concat([pd.DataFrame(a), pd.DataFrame(b), pd.DataFrame(c)], axis=1)